In [13]:
import pandas as pd

In [16]:
bank = pd.read_csv('Data/bank3.csv')

In [12]:
bank.head()

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,A,B,C,D,E
0,409000611074,2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1000000.0,indiaforensic,0.0,0.0,0.0,0.0
1,409000611074,2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2000000.0,indiaforensic,0.0,0.0,0.0,0.0
2,409000611074,2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2500000.0,fdrl,0.0,internal,0.0,0.0
3,409000611074,2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5500000.0,indiaforensic,0.0,0.0,0.0,0.0
4,409000611074,2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6000000.0,fdrl,0.0,internal,0.0,0.0


In [3]:
bank.drop(["A", "B", "C", "D", "E"], axis=1, inplace=True)
bank.drop(["CHQ.NO."], axis=1, inplace=True)

In [4]:
# Value date : Date of completion
# Date : Inititation
bank.head()

,Account No,DATE,TRANSACTION DETAILS,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT
0,409000611074,2017-06-29,TRF FROM Indiaforensic SERVICES,2017-06-29,NaN,1000000.0,1000000.0
1,409000611074,2017-07-05,TRF FROM Indiaforensic SERVICES,2017-07-05,NaN,1000000.0,2000000.0
2,409000611074,2017-07-18,FDRL/INTERNAL FUND TRANSFE,2017-07-18,NaN,500000.0,2500000.0
3,409000611074,2017-08-01,TRF FRM Indiaforensic SERVICES,2017-08-01,NaN,3000000.0,5500000.0
4,409000611074,2017-08-16,FDRL/INTERNAL FUND TRANSFE,2017-08-16,NaN,500000.0,6000000.0


In [5]:
df_acc1 = bank[bank["Account No"] == 409000611074]

In [19]:
# df_acc1.head()
df_acc1.to_html('temp2.html', notebook=True)

In [7]:
from fuzzywuzzy import fuzz

# Returns a dataframe object consisting of matching narrations 
def seq_match(df_temp, trans_details):
    
    df_temp.reset_index(inplace=True, drop=True)

    df_res = []
    for i, (x, y) in enumerate(df_temp.iterrows()):
        # Considered threshold : 30
        if fuzz.ratio(trans_details, df_temp.loc[i, "TRANSACTION DETAILS"]) > 30:
            df_res.append(df_temp.loc[i]) 
            
    return pd.DataFrame(df_res) 


# For testing purposes, considered a sample transaction narration
# trans_details = "TRF FROM Indiaforensic SERVICES"
# res = seq_match(df_acc1, trans_details)
# res.head()

/home/apoorv/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [30]:
# ###################### Testing ######################
# for x in res['TRANSACTION DETAILS']:
#     print(x)

account_list = bank["Account No"].unique()
print(account_list)
# df_acc1 = bank[bank["Account No"] == 409000362497]
# df_acc1.reset_index(inplace=True)

[409000611074 409000493201 409000425051 409000405747 409000438611
 409000493210 409000438620      1196711      1196428 409000362497]


In [78]:
# Subset sum algorithm
def subsetsum(array, num):

    if num == 0 or num < 1:
        return []
    elif len(array) == 0:
        return []
    else:
        if array[0] == num:
            return [array[0]]
        else:
            with_v = subsetsum(array[1:],(num - array[0])) 
            if with_v:
                return [array[0]] + with_v
            else:

                return subsetsum(array[1:],num)

In [8]:
# ###################### Detecting bogus transactions ######################

# [409000611074 409000493201 409000425051 409000405747 409000438611
#  409000493210 409000438620      1196711      1196428 409000362497]

ctr = 0
# 1. 409000611074
# 2. 409000493201
# 3. 409000425051
# 4. 409000405747
# 5. 409000438611 - Bogus
# 6. 409000493210 - Bogus
# 7. 409000438620 - Bogus
# 8. 1196711 - Bogus bc
# 9. 1196428 - Bogus
# 10. 409000362497 - Bogus

df_acc1 = bank[bank["Account No"] == 409000362497]

for (index, row) in df_acc1.iterrows():
    
    if index % 1000 == 0:
        print(index)
    
    if row["DEPOSIT AMT"] >= 0:
        acc_no = row["Account No"]
        date = row["VALUE DATE"]
        deposit_amt = row["DEPOSIT AMT"]
        trans_details = row["TRANSACTION DETAILS"]
        
        df_temp =\
            df_acc1[(df_acc1["Account No"] == acc_no) & (df_acc1["VALUE DATE"] == date) & (df_acc1["WITHDRAWAL AMT"] >= 0)]
    
        df_temp_res = seq_match(df_temp, trans_details)        
        
        if not df_temp_res.empty:        
            withdrawal_amt = list(df_temp_res["WITHDRAWAL AMT"])
            subset =  subsetsum(withdrawal_amt, deposit_amt)
            if len(subset) > 0:
                print("***************************************")
                print(f"Deposit Amount {deposit_amt}")
                print(f"Withdrawal Amount : {subset}")
                print(f"Date {date}")
                print(f"Transaction Details {trans_details}")
                print("***************************************")                

            ctr += 1


87000
88000
***************************************
Deposit Amount 412421.0
Withdrawal Amount : [412421.0]
Date 2015-10-19
Transaction Details P2P NEFT SETTLEMENT  INTE
***************************************
89000
90000
91000
***************************************
Deposit Amount 33652.47
Withdrawal Amount : [33652.47]
Date 2016-01-30
Transaction Details IMPS IW 30012016 4C
***************************************
***************************************
Deposit Amount 26350979.59
Withdrawal Amount : [26350979.59]
Date 2016-01-30
Transaction Details REV IMPS OW 30012016 4C
***************************************
***************************************
Deposit Amount 78792.06
Withdrawal Amount : [78792.06]
Date 2016-02-03
Transaction Details IMPS IW 01022016 1C
***************************************
***************************************
Deposit Amount 85.88
Withdrawal Amount : [85.88]
Date 2016-02-03
Transaction Details IMPS PENALTY 01022016 1C
***************************************


116000


### Checking out different string sequence matching tools

In [17]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [29]:
similar(bank.loc[116172]["TRANSACTION DETAILS"], bank.loc[116174]["TRANSACTION DETAILS"])

0.7857142857142857

In [31]:
from fuzzywuzzy import fuzz

print(fuzz.ratio(bank.loc[116172]["TRANSACTION DETAILS"], bank.loc[116174]["TRANSACTION DETAILS"]))

79
